In [ ]:
import telebot
from telebot import types
from urllib.request import urlopen
import json
import pandas as pd
import requests
import ast


token = '2024300855:AAEtuoI4_1H9TF76Dzho_wwtlaA2Qj3vtuU'
bot = telebot.TeleBot(token)

users = {}

In [ ]:
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, f'Привет, {message.from_user.username}!')
    if message.from_user.username not in users.keys():
        bot.send_message(message.chat.id, f'Вы новый пользователь. Чтобы начать, используйте команды /get_tasks для просмотра возможных задач и моделей')
        users[message.from_user.username] = None
    else:
        bot.send_message(message.chat.id, f'Чтобы воспользоваться уже обученной моделью используйте команду /get_preds)


@bot.message_handler(commands=['get_tasks'])
def choose_task(message):
    
    markup = types.ReplyKeyboardMarkup()

    resp = requests.get('http://127.0.0.1:5000/api/model_tasks').text
    tasks = resp.split(', ')
    len_taks = len(tasks)
    if len_taks == 2:
        item_1, item_2 = [types.KeyboardButton(x) for x in tasks]
        markup.row(item_1, item_2)
        bot.send_message(message.chat.id, "Какую задачу машинного обучения будем решать?", reply_markup=markup)
        bot.register_next_step_handler(message, choose_model)
    elif len_taks == 3:
        item_1, item_2, item_3 = [types.KeyboardButton(x) for x in tasks]
        markup.row(item_1, item_2, item_3)
        bot.send_message(message.chat.id, "Какую задачу машинного обучения будем решать?", reply_markup=markup)
        bot.register_next_step_handler(message, choose_model)
    
@bot.message_handler(commands=['get_types'])
def choose_model(message):
    
    global task_
    task_ = message.text
    
    markup = types.ReplyKeyboardMarkup()

    resp = requests.get(f'http://127.0.0.1:5000/api/model_types/{task_}').text
    model_types = resp.split(', ')
    len_types = len(model_types)
    if len_types == 2:
        item_1, item_2 = [types.KeyboardButton(x) for x in model_types]
        markup.row(item_1, item_2)
        bot.send_message(message.chat.id, "Какую модель будем обучать?", reply_markup=markup)
        bot.register_next_step_handler(message, get_data)
    elif len_types == 3:
        item_1, item_2, item_3 = [types.KeyboardButton(x) for x in model_types]
        markup.row(item_1, item_2, item_3)
        bot.send_message(message.chat.id, "Какую модель будем обучать?", reply_markup=markup)
        bot.register_next_step_handler(message, get_data)

def get_data(message):
    
    global model_type_ 
    model_type_ = message.text
    
    bot.send_message(message.chat.id, f'{message.from_user.username}, пожалуйста, загрузите тренировочную выборку в csv файле')
    bot.register_next_step_handler(message, get_dataframe)

def get_dataframe(message):
    url = f'https://api.telegram.org/bot{token}/getFile?file_id={message.document.file_id}'
    response = urlopen(url)
    data_json = json.loads(response.read())
    url_2 = f"https://api.telegram.org/file/bot{token}/{data_json['result']['file_path']}" # здесь наша CSV
    global df
    df = pd.read_csv(url_2)

    bot.send_message(message.chat.id, f'Выберите целевую переменную из предложенного ниже списка:')
    answer = ''
    for col in df.columns: 
        answer += "• " + col + '\n'
    bot.send_message(message.chat.id, answer)  
    bot.register_next_step_handler(message, split_df)


def split_df(message):
    global y
    global X
    global col_num_X
    global target_col
    
    target_col = message.text
    y = pd.DataFrame(df[target_col])
    X = df.drop(columns=[target_col]) # данные есть, готовы отправлять сигнал к нашему API
    col_num_X = X.shape[1]
    bot.send_message(message.chat.id, f'Данные успешно обработаны')
    
    markup = types.ReplyKeyboardMarkup()
    item_1 = types.KeyboardButton('Да')
    item_2 = types.KeyboardButton('Нет')
    markup.row(item_1, item_2)
    bot.send_message(message.chat.id, "Вы хотите использовать стандартный набор гиперпараметров?", reply_markup=markup)
    
    bot.register_next_step_handler(message, get_params)


def get_params(message):
    global params_
    params_ = {}
    if message.text == 'Да':
        bot.send_message(message.chat.id, f'Введите название вашей модели')
        bot.register_next_step_handler(message, get_model_name_and_fit)
    else: 
        bot.send_message(message.chat.id, f'Напишите необходимые гиперпараметры в формате python-словаря.')
        bot.register_next_step_handler(message, parse_params)
        

def parse_params(message):
    global params_
    params_ = ast.literal_eval(message.text)
    bot.send_message(message.chat.id, f'Введите название вашей модели')
    bot.register_next_step_handler(message, get_model_name_and_fit)     
        
def get_model_name_and_fit(message):
    global name_
    name_ = message.text
    
    model_json = {'task': task_,
                  'model': model_type_,
                  'model_name': name_,
                  'X': X.to_dict(),
                  'y': y.to_dict(),
                  'params': params_}
    resp = requests.put(f'http://127.0.0.1:5000/api/fit', json=model_json).text
    bot.send_message(message.chat.id, resp)
    
    if resp[:5] == 'Model':
        markup = types.ReplyKeyboardMarkup()
        item_1 = types.KeyboardButton('Да')
        item_2 = types.KeyboardButton('Нет')
        markup.row(item_1, item_2)
        bot.send_message(message.chat.id, "Чтобы сделать предсказание модели используйте команду /get_preds")
    else: 
        pass

@bot.message_handler(commands=['get_preds'])
def get_model_name(message):
    bot.send_message(message.chat.id, "Напишите название уже обученной модели")
    bot.register_next_step_handler(message, get_test_set)
    
def get_test_set(message):
    global name_model_pred
    name_model_pred = message.text
    
    global X_test
    
    bot.send_message(message.chat.id, "Загрузите файл с тестовыми данными")
    bot.register_next_step_handler(message, preds)
    
def preds(message):
    
    url = f'https://api.telegram.org/bot{token}/getFile?file_id={message.document.file_id}'
    response = urlopen(url)
    data_json = json.loads(response.read())
    url_2 = f"https://api.telegram.org/file/bot{token}/{data_json['result']['file_path']}" # здесь наша CSV
    global df_test
    df_test = pd.read_csv(url_2)
    
    if len(df_test.columns) == col_num_X:
        pred_json = {'model_name': name_model_pred,
                     'X_test': df_test.to_dict()}
        resp = requests.put(f'http://127.0.0.1:5000/api/predict', json=pred_json).text
        print(resp)
    else: 
        try: 
            df_test = df_test.drop(columns=[target_col])
            pred_json = {'model_name': name_model_pred,
                     'X_test': df_test.to_dict()}
            resp = requests.put(f'http://127.0.0.1:5000/api/predict', json=pred_json).json()
            predictions = pd.DataFrame()
            predictions['preds'] = list(resp.values())
            predictions.index = list(resp.keys()) 
            predictions.to_csv('preds.csv')
            doc = open('preds.csv', 'rb')
            bot.send_document(message.chat.id, doc)  
            
        except Exception:
            bot.send_message(message.chat.id, "Не вышло :(")
        

bot.infinity_polling()